In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable

# === 1. Load feature data ===
X = pd.read_csv("dataset/final_fire_dataset2.csv")
y = X['fire_occurred'].values.astype(np.float32)
X = X.drop(columns=['fire_occurred'])

# === 2. Generate modulator labels (pseudo-label: 1.0 = trust, 0.5 = suppress) ===
# These are *not* binary fire labels — these are "how confident should we be"
def generate_trust_score(row):
    temp = row['temperature']
    humidity = row['humidity']
    wind = row['wind_speed']
    veg = row['vegetation_index']

    # Normalize key ranges
    temp_factor = np.clip((temp - 295) / 15, 0, 1)        # 295–310 K
    humidity_factor = np.clip((30 - humidity) / 30, 0, 1) # Lower humidity → higher risk
    wind_factor = np.clip((wind - 5) / 10, 0, 1)          # Above 5 m/s helps
    veg_factor = np.clip((veg - 0.2) / 0.8, 0, 1)         # Higher vegetation = more fuel

    # Weighted blend
    composite = (0.4 * temp_factor +
                 0.2 * humidity_factor +
                 0.2 * wind_factor +
                 0.2 * veg_factor)

    # Final trust modulation range: [0.5, 1.5]
    return 0.5 + composite

mod_labels = X.apply(generate_trust_score, axis=1).values.astype(np.float32)

# === 3. Normalize inputs ===
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 4. Define FireTrustNet ===
@register_keras_serializable()
def firetrust_activation(x):
    return 0.5 + tf.sigmoid(x)  # output in range [0.5, 1.5]

input_layer = layers.Input(shape=(5,))
z = layers.Dense(16, activation='relu')(input_layer)
z = layers.Dense(8, activation='relu')(z)
output = layers.Dense(1, activation=firetrust_activation)(z)

firetrustnet = models.Model(inputs=input_layer, outputs=output)
firetrustnet.compile(optimizer='adam', loss='mse')
firetrustnet.summary()

# === 5. Train FireTrustNet ===
firetrustnet.fit(X_scaled, mod_labels, batch_size=32, epochs=10, validation_split=0.2)

# === 6. Save everything ===
firetrustnet.save("FireTrustNet.h5")
import joblib
joblib.dump(scaler, "firetrust_scaler.pkl")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0078 - val_loss: 6.9959e-04
Epoch 2/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 5.1378e-04 - val_loss: 2.4366e-04
Epoch 3/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2.0198e-04 - val_loss: 1.3577e-04
Epoch 4/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.2036e-04 - val_loss: 8.1197e-05
Epoch 5/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 8.4336e-05 - val_loss: 6.3300e-05
Epoch 6/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 6.8443e-05 - val_loss: 5.4133e-05
Epoch 7/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 5.4473e-05 - val_loss: 5.3971e-05
Epoch 8/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 4.4969e-05 - val_loss: 2.9369e-05
Epoch 9/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3.5460e-05 - val_loss: 6.2119e-05
Epoch 10/10
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 4.0504e-05 - val_loss: 5.0847e-05


['firetrust_scaler.pkl']